# <div class="alert alert-block alert-success">About this notebook</div>

In this notebook, I tried to see similarity of foods.

I focus on ingredients of food. This is because if we use same ingredient, the results should be naturally similar. Of cource, even if the ingredients are the same, different processing methods could result in a completely different dish, but let's turn a blind eye to that in this case.

But by putting this simple hypothesis, I found interesting result, so I'll share that.

<u>update information</u>

version1: first release

version2: fixed calculation method of ingredients word.

## <div class="alert alert-block alert-info">Preperation</div>

## Load liblraries

In [ ]:
import collections
import json
import os

import folium
import matplotlib.pyplot as plt
import nltk
import numpy as np
import re
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import umap
from wordcloud import WordCloud

In [ ]:
plt.style.use("Solarize_Light2")

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Data overviewing

Let's load dataset. There are one csv data.

In [ ]:
!ls ../input/indian-food-101

In [ ]:
df = pd.read_csv("/kaggle/input/indian-food-101/indian_food.csv")

In [ ]:
df.head()

In [ ]:
df.shape

This data has 9 columns and 255 records.

Let's see data type and wheather there are null or not.

In [ ]:
df.info()

There are no null. prep_time and cook_time are int and others are string. 

# <div class="alert alert-block alert-info">EDA</div>

### Dish

Let's see data overview and its' trends.

In [ ]:
print("There are ", len(set(df['name'])), "dish")

There doesn't seem to be any overlap in the dishes.

In [ ]:
df_diet =  df[["diet"]].copy()
df_diet["count"] = 1
df_diet = df_diet.groupby("diet").count()
df_diet.head()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,6), gridspec_kw=dict(wspace=0.1, hspace=0.6))
fig.suptitle("Analisys of diet", fontsize=15)

g_diet = sns.countplot(data=df, x="diet", order = df['diet'].value_counts().index,
                       ax=axes[0])
g_diet.set_title("diet countplot")

g_diet_pie = df_diet.plot.pie(y="count", ax=axes[1])
g_diet_pie.set_title("diet pie plot")

Most of them seem to be vegitarian.

Let's see on view of rate. I'll count them.

Looking at the pie chart, the ratios were more intuitive. We can see that it's almost 90% vegetarian.

### flavor_profile

flavor_profile represents dish flaver. Let's see them.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,6), gridspec_kw=dict(wspace=0.1, hspace=0.6))
fig.suptitle("Analisys of flavor profile", fontsize=15)

g_flavor_profile = sns.countplot(data=df, x="flavor_profile", ax=axes[0], 
                                 order = df['flavor_profile'].value_counts().index)
g_flavor_profile.set_title("flavor profile countplot")

df_flavor_profile =  df[["flavor_profile"]].copy()
df_flavor_profile["count"] = 1
df_flavor_profile = df_flavor_profile.groupby("flavor_profile").count()
g_flavor_profile_pie = df_flavor_profile.plot.pie(y="count", ax=axes[1])
g_flavor_profile_pie.set_title("flavor profile pie plot")

-1 in any of the columns indicates NaN value.

The most common flavor is spicy. The next most common is sweet. We can see that these two make up roughly 80% or more of the total.

### course

Cource represents dish's type in cource.

In [ ]:
g_course = sns.countplot(data=df, x="course", order = df['course'].value_counts().index)
g_course.set_title("course countplot")

The most common is main cource. The next most common is dessert. The third is snack. Also, there is a little starter.

### region

region represents witch region's dish. The characteristics of the dishes should be influenced by cooperation in the region.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,6), gridspec_kw=dict(wspace=0.1, hspace=0.6))
fig.suptitle("Analisys of region", fontsize=15)

g_region = sns.countplot(data=df, x="region", order = df['region'].value_counts().index, ax=axes[0])
g_region.set_title("region countplot")

g_region = sns.countplot(data=df, x="region", hue="flavor_profile", ax=axes[1],
                         order = df['region'].value_counts().index)
g_region.legend_._loc = 1
g_region.set_title("taste countplot per region")

Let's visualize the flavor_profile for each region to try it out. It's almost the same, but most of the regions have a lot of spicy profiles, while East has a lot of sweet profiles. Also, sour is concentrated in West.

### state

region represents which state's dish.

In [ ]:
plt.figure(figsize=(20, 10))
g_state = sns.countplot(data=df, x="state",order = df['state'].value_counts().index)
g_state.set_xticklabels(g_state.get_xticklabels(), rotation=45)
g_state.set_title("state countplot")

The states seem to be somewhat scattered, but they tend to be concentrated in the first six or so states. Also there is quite a lot of null.

### prep_time and cook_time

These columns represent preperation and cooking time.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,6), gridspec_kw=dict(wspace=0.1, hspace=0.6))
fig.suptitle("Analisys of prep&cook time", fontsize=15)

g_prep_time = sns.distplot(df["prep_time"], ax=axes[0])
g_prep_time.set_title("prep_time countplot")

g_cook_time = sns.distplot(df["cook_time"], ax=axes[1])
g_cook_time.set_title("cook_time countplot")

Most of the foods seem to be completed preparing and cooking in less than 100 minutes.

But, there are very elaborate dish... I would love to try such painstakingly prepared food.

### ingredients

ingredients represents what the food is made of. This is sentence, so I'll parse them to words and count frequency.

In [ ]:
all_words = []
for i in range(len(df)):
    txt =  df["ingredients"][i]
    #txt =  txt.replace(', ', ',').lower()
    #all_words += [ word for word in re.split('[,.]',txt) ]
    all_words += [word.lower() for word in nltk.word_tokenize(txt) if not word in ['.', ',']]
    
word_freq = collections.Counter(all_words)
W = WordCloud().fit_words(word_freq)

And let's turn it into a word cloud.

In [ ]:
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(W)
plt.axis('off')
plt.show()

Flour, dal and sugar are noticeable in cloud. 

# <div class="alert alert-block alert-info">Analysis of similarity</div>

Now, I focus on word included in ingredients. This is because if we use same ingredient, the results should be naturally similar.

First, we got all ingredient words. They are following.

In [ ]:
words = np.array(list(word_freq.keys()))
words

From all words and ingredients of each food, I create "ingredients vector". This vector represents which ingredients the food has. Length of this vector is lenght of list of all ingredient words. If food includes "red pepper" as ingredient, the element of the vector corresponding to "red pepper" is 1. If not, it's zero.

In [ ]:
def gen_ingredients_vector(ingredients):
    ingredients_vec = np.zeros(words.shape)
    ingredients = set([word.lower() for word in nltk.word_tokenize(ingredients) if not word in ['.', ',']])
    for ingredient in ingredients:
        idx = np.where(words == ingredient)
        ingredients_vec[idx] = 1
    return ingredients_vec.tolist()

df["ingredients_vec"] = df["ingredients"].map(gen_ingredients_vector)

df.head()

## Cosine similarity

I use cosine similarity to calculate similarity of ingredients vectors. Cosine similarity often used to calculate sentence similarity. In fact, ingredients vectors represent sentence, so it should be a good match. If cosine similarity between two foods is high, we can say the foods are similar. 

![cosine similarity](https://wikimedia.org/api/rest_v1/media/math/render/svg/1d94e5903f7936d3c131e040ef2c51b473dd071d)

For more detail of cosine similarity, please refer following:

https://en.wikipedia.org/wiki/Cosine_similarity

In [ ]:
# To input heatmap, I made ingredients_vecs.
ingredients_vecs = []
for i in range(len(df)):
    ingredients_vecs.append(df["ingredients_vec"][i])
    
ingredients_vecs = np.array(ingredients_vecs)

In [ ]:
ingredients_vecs

In [ ]:
cos_matrix = cosine_similarity(ingredients_vecs, ingredients_vecs)

In [ ]:
plt.figure(figsize=(20, 20))
ax = sns.heatmap(cos_matrix)
ax.set_title("cosine_similarity of ingredients_vectors")

We can't understand the details because plot is so detailed 😅, but we can see that there are about 5 blocks of high cosine similarity near the diagonal elements.



For example, 0 - 66th (as you can see, this numbers correspond to index of "df")  ingredients vectors have high cosine similarity each others. 

Think in the same way, we reckon 66th - 108th, 109th - 159th, 160th - 222th and 223th - 255th as blocks. (Note that there is no statistical basis for this, it's just an rough estimate.)

Let's find out exactly how similar they really are!

### Following two sweets are similar. Their cosine similarity is high.

In [ ]:
df.iloc[[10,12]]

In [ ]:
cosine_similarity([ingredients_vecs[10]], [ingredients_vecs[12]])

Laddu is a sphere-shaped sweet originating from the Indian subcontinent. Nankhatai are shortbread biscuits, originating from the Indian subcontinent, popular in Northern India and Pakistan. This two sweets are similar.

- Laddu

![Laddu](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Laddu-Janakpur-Community_Outreach-City_Tour_Day-4-6751.jpg/220px-Laddu-Janakpur-Community_Outreach-City_Tour_Day-4-6751.jpg)

- Nankhatai

![Nankhatai](https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Nankhatai.jpg/220px-Nankhatai.jpg)

(Explanations and pictures are from https://en.wikipedia.org/wiki/)

### Following two sweets are not similar. Their cosine similarity is low.

In [ ]:
df.iloc[[10,56]]

In [ ]:
cosine_similarity([ingredients_vecs[10]], [ingredients_vecs[56]])

Basundi is an Indian sweet in Gujarat, Maharashtra , Telangana, Tamil Nadu and Karnataka. By following pictures, you can found it's certainly not similar.

- Laddu

![Laddu](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Laddu-Janakpur-Community_Outreach-City_Tour_Day-4-6751.jpg/220px-Laddu-Janakpur-Community_Outreach-City_Tour_Day-4-6751.jpg)

- Basundi

![Basundi](https://upload.wikimedia.org/wikipedia/commons/thumb/3/39/Basundi_Indian_Dessert_Mithai_Sweets.jpg/220px-Basundi_Indian_Dessert_Mithai_Sweets.jpg)

(Explanations and pictures are from https://en.wikipedia.org/wiki/)

## Demonstrating the validity of the vectors by kmeans and umap.

I also try to demonstrate the validity of ingredients vector by kmeans and umap.

By cosine similarity heatmap, we found that there are five gruops, so I'll define the groups by kmeans. (Of cource, these cluster doesn't necessarily match the five group, though.)

And using umap, I'll confirm that we can separete ingredients vector to the clusters in low dimension.

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(ingredients_vecs)

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(ingredients_vecs)

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=kmeans.labels_)
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of ingredients vectors', fontsize=15)

As you can see, some of it is mixed, but you can see that the foods are coming together relatively nicely. So we may say that ingredients vector represents simmilarity of foods nicely.

## Similarity search:

To find out similar dishes region-wise or state-wise based on list of ingredients, I made get_cosine_similarity_heatmap function. You can generate heatmap filtered by region and state. If you can see high similar cells on heatmap, you can search high similar dishes.

Let's search!

In [ ]:
def get_cosine_similarity_heatmap(df,state=None, region=None):
    """
    Visualize cosine similarity heatmap of ingredients vector.
    And return filtered dataframe.
    """
    
    if state==None and region==None:
        df_filtered = df
    elif state!=None:
        df_filtered = df[df["state"]==state]
    elif region!=None:
        df_filtered = df[df["region"]==region]
    else:
        df_filtered = df[df["state"]==state & df["region"]==region]
    
    #ingredients_vecs = []
    
    ingredients_vecs = [vec for vec in df_filtered["ingredients_vec"]]
    #for i in range(len(df_filtered)):
    #    ingredients_vecs.append(df_filtered["ingredients_vec"][i])
    
    ingredients_vecs = np.array(ingredients_vecs)
    cos_matrix = cosine_similarity(ingredients_vecs, ingredients_vecs)
    
    plt.figure(figsize=(15, 15))
    ax = sns.heatmap(cos_matrix)
    ax.set_title(f"cosine_similarity of ingredients_vectors in {state} and {region}")
    
    return df_filtered.reset_index()

In [ ]:
df_WestBengal_East = get_cosine_similarity_heatmap(df, "West Bengal", "East")

As you can see heatmap, index 5 and 8 are similar.

Let's check their ingredients.



In [ ]:
df_WestBengal_East.iloc[[5,8]]

Index 5 is Ledikeni and index 9 is Pantua. Luckily, wikipedia clearly states that the two dishes are similar.

> Pantua is very similar to the cheese-based fried sweet ledikeni.

![Pantua](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/Pantua_-_Kolkata_2011-09-20_5431.JPG/220px-Pantua_-_Kolkata_2011-09-20_5431.JPG)

Pitcure and explanation are from https://en.wikipedia.org/wiki/Pantua

## <div class="alert alert-block alert-success">Concludion</div>

I made word vector called  ingredients vector from dish's ingredients.

With cosine similarity, I can find similar dishes region-wise or state-wise.